<div style="display: block; height: 500px; overflow:hidden;position: relative; padding-bottom:50px">
     <img src="https://imgur.com/VF9rSJb.jpg" style="position: absolute;top: 0px;border-radius: 20px; ">
</div>

# 1. Imports

In [ ]:
#
import numpy as np
import pandas as pd 
import random

# image
from PIL import Image

# visu
import matplotlib.pyplot as plt

# folder
import os
import glob

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

#tensorflow
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 2. Loading image data

There are five flower categories. The images are loaded in a numpy array as matrix and associated categories are loaded in an independent array.

In [ ]:
categories = ["dandelion", "daisy", "sunflower", "tulip", "rose"]

We resize images so they all have the same width and height. We select the width as the mean width of all images and the height as the mean height of all images.

In [ ]:
%%time
#
images_shapes = {"height": [], "width": []}
#
for cat in categories:
    filelist = glob.glob('./data/' + cat + '/*.jpg')
    for fname in filelist:
        images_shapes["height"].append(np.array(Image.open(fname)).shape[0])
        images_shapes["width"].append(np.array(Image.open(fname)).shape[1])

In [ ]:
display("Average height: " + str(int(np.mean(images_shapes["height"]))))
display("Average width: " + str(int(np.mean(images_shapes["width"]))))

Because of memory limitation in Kaggle, keeping 338 x 253 is not possible. Let's divide the height and width by two.

In [ ]:
im_width = int(338/2)
im_height = int(253/2)

In [ ]:
display("Used height: " + str(im_height))
display("Used width: " + str(im_width))

Now images are loaded and resized with a width of 169, and a height of 126 and stored in the numpy array:

In [ ]:
data = []
target = []

In [ ]:
%%time
for cat in categories:
    filelist = glob.glob('./data/' + cat + '/*.jpg')
    target.extend([cat for _ in filelist])
    data.extend([np.array(Image.open(fname).resize((im_width, im_height))) for fname in filelist])
#
data_array = np.stack(data, axis=0)

So we have 4317 tensor images of width 169 and height 126, each pixel being defined by three colors R, G, B:

In [ ]:
data_array.shape

We can check by random images that each of them have the same size:

In [ ]:
fig = plt.figure(figsize=(20,15))
gs = fig.add_gridspec(4, 4)
#
for line in range(0, 3):
    for row in range(0, 3):
        num_image = random.randint(0, data_array.shape[0])
        ax = fig.add_subplot(gs[line, row])
        ax.axis('off');
        ax.set_title(target[num_image])
        ax.imshow(data_array[num_image]);

# 3. Train test split

As indicated in the instructions, we use the random seed 43 and a test set size of 20% of the dataset. Moreover, we use the parameter `stratify`set to `target` so that the class repartition is maintained

In [ ]:
pd.DataFrame(target).value_counts()/len(target)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_array, np.array(target), random_state=43, test_size=0.2, stratify=target)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
pd.DataFrame(y_train).value_counts()/len(y_train)

In [ ]:
pd.DataFrame(y_test).value_counts()/len(y_test)

# 4. Preparing the data

## Normalization
To ease the convergence of the algorithm, it is usefull to normalize the data. See here what are the maximum and minimum values in the data, and normalize it accordingly (the resulting image intensities should be between 0 and 1).

In [ ]:
print(X_train.max())
print(X_train.min())

In [ ]:
X_test_norm = np.round((X_test/255), 3).copy()
X_train_norm = np.round((X_train/255), 3).copy()

Here again, we can check the normalised pictures randomly:

In [ ]:
fig = plt.figure(figsize=(20,15))
gs = fig.add_gridspec(4, 4)
#
for line in range(0, 3):
    for row in range(0, 3):
        num_image = random.randint(0, X_train_norm.shape[0])
        ax = fig.add_subplot(gs[line, row])
        ax.axis('off');
        ax.set_title(y_train[num_image])
        ax.imshow(X_train_norm[num_image]);

## Target encoding

Here we convert targets. First, from string to numerical values, each category becoming an integer, from 0 to 4 (as there are five different flower categories):

In [ ]:
display(np.array(y_train).shape)
display(np.unique(y_train))
display(np.array(y_test).shape)
display(np.unique(y_test))

Fitting the encoder on train set:

In [ ]:
encoder = LabelEncoder().fit(y_train)

Applying on both train and test set:

In [ ]:
y_train_cat = encoder.transform(y_train)
y_test_cat = encoder.transform(y_test)

And now, we convert the result to one-hot encoded target so that they can be used to train a classification neural network. We use `to_categorical` from tensorflow library:

In [ ]:
y_train_oh = to_categorical(y_train_cat)
y_test_oh = to_categorical(y_test_cat)

In [ ]:
pd.DataFrame(y_test_oh).head()

# 5. Convolutionnal neural network

Now, let's define the Convolutional Neural Network. 

The CNN that is composed of:
- a Conv2D layer with 32 filters, a kernel size of (3, 3), the relu activation function, a padding equal to `same` and the correct `input_shape`
- a MaxPooling2D layer with a pool size of (2, 2)
- a Conv2D layer with 64 filters, a kernel size of (3, 3), the relu activation function, and a padding equal to `same`
- a MaxPooling2D layer with a pool size of (2, 2)
- a Conv2D layer with 128 filters, a kernel size of (3, 3), the relu activation function, and a padding equal to `same`
- a MaxPooling2D layer with a pool size of (3, 3)
- a Flatten layer
- a dense function with 120 neurons with the `relu` activation function
- a dense function with 60 neurons with the `relu` activation function
- a dropout layer (with a rate of 0.5), to regularize the network
- a dense function related to your task: multiclassification

In [ ]:
def initialize_model():
    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3), activation="relu", input_shape=(im_height, im_width, 3), padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(64, (3, 3), activation="relu", padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2, 2)))
    model.add(layers.Conv2D(128, (3, 3), activation="relu", padding='same'))
    model.add(layers.MaxPool2D(pool_size=(3, 3)))
    model.add(layers.Flatten())
    model.add(layers.Dense(120, activation='relu'))
    model.add(layers.Dense(60, activation='relu'))
    model.add(layers.Dropout(rate=0.2))
    model.add(layers.Dense(5, activation='softmax'))

    return model

In [ ]:
model = initialize_model()
model.summary()

In [ ]:
def compile_model(model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics="accuracy")
    return model

Here I set an early stopping after 5 epochs and set the parameter `restore_best_weights` to `True` so that the weights of best score on monitored metric - here `val_accuracy` (accuracy on test set) - are restored when training stops. This way the model has the best accuracy possible on unseen data.

In [ ]:
model = initialize_model()
model = compile_model(model)
es = EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True)

#model = initialize_model()
history = model.fit(X_train_norm, y_train_oh,
                    batch_size=16,
                    epochs=50,
                    validation_data=(X_test_norm, y_test_oh),
                    callbacks=[es])

# 6. Results

In [ ]:
def plot_history(history, title='', axs=None, exp_name=""):
    if axs is not None:
        ax1, ax2 = axs
    else:
        f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))
    
    if len(exp_name) > 0 and exp_name[0] != '_':
        exp_name = '_' + exp_name
    ax1.plot(history.history['loss'], label='train' + exp_name)
    ax1.plot(history.history['val_loss'], label='val' + exp_name)
    ax1.set_ylim(0., 2.2)
    ax1.set_title('loss')
    ax1.legend()

    ax2.plot(history.history['accuracy'], label='train accuracy'  + exp_name)
    ax2.plot(history.history['val_accuracy'], label='val accuracy'  + exp_name)
    ax2.set_ylim(0.25, 1.)
    ax2.set_title('Accuracy')
    ax2.legend()
    return (ax1, ax2)

plot_history(history, title='', axs=None, exp_name="");

In [ ]:
model.evaluate(X_test_norm, y_test_oh, verbose=0)

So we have an accuracy on unseen data of almost 70%.

# 7. Data augmentation

We try to improve the model accuracy by using the data augmentation. It consists in applying little transformation to input images without changing its label.

For this, we use `ImageDataGenerator` from tensorflow. It will generate images a little bit different from an original image so that it will be like the algorithm is training on more data

In [ ]:
datagen = ImageDataGenerator(featurewise_center=False,
                             featurewise_std_normalization=False,
                             rotation_range=10,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             horizontal_flip=True,
                             vertical_flip=True,
                             zoom_range=(0.8, 1.2),) 
#
datagen.fit(X_train_norm)

Here after, we can look at the original image, and the same image after its small transformation:

In [ ]:
X_augmented = datagen.flow(X_train_norm, shuffle=False, batch_size=1)

for i, (raw_image, augmented_image) in enumerate(zip(X_train_norm, X_augmented)):
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(6, 2))
    ax1.imshow(raw_image)
    ax2.imshow(augmented_image[0])
    plt.show()
    
    if i > 10:
        break

Let's train the model with this improvment:

In [ ]:
model_aug = initialize_model()
model_aug = compile_model(model_aug)
train_flow = datagen.flow(X_train_norm, y_train_oh, batch_size=32)
es = EarlyStopping(patience=5, monitor='val_accuracy', restore_best_weights=True)

#model = initialize_model()
history_aug = model_aug.fit(train_flow,
                            epochs=50,
                            validation_data=(X_test_norm, y_test_oh),
                            callbacks=[es])

In [ ]:
plot_history(history_aug, title='', axs=None, exp_name="");

In [ ]:
model_aug.evaluate(X_test_norm, y_test_oh, verbose=0)

We obtain almost 10% more accuracy on unseen data compared to the initial model!

In [ ]:
axs = plot_history(history_aug, exp_name='data_augmentation')
plot_history(history ,axs=axs, exp_name='baseline')
plt.show()

<b>Thank you for reading 🙂</b> <br>if you have any remarks about the content of this notebook, if there are some mistakes or if you have suggestions for improvment, please feel free to comment